In [1]:
%%writefile daemon.json
{
    "runtimes": {
        "nvidia": {
            "path": "nvidia-container-runtime",
            "runtimeArgs": []
        }
    },
    "data-root": "/home/ec2-user/SageMaker/docker"
}

Writing daemon.json


In [2]:
%%bash
sudo systemctl stop docker
sudo cp daemon.json /etc/docker/daemon.json
sudo cp -rp /var/lib/docker /home/ec2-user/SageMaker/docker
sudo systemctl start docker

  docker.socket


In [34]:
!docker build --quiet --tag api-layer:latest .

sha256:7a5dfc98d5bbfecdb3881f46d37c81aecdff35c99b83a89f53f05215f8b36314


In [35]:
!docker run --name test-api-layer -d -p 8001:8001 -e AWS_REGION=us-west-2 -e AWS_DEFAULT_REGION=us-west-2 api-layer:latest --host 0.0.0.0 --port 8001 --table-name api-layer-ddb

7ac490de66723e02a06bf48a15ac1c64ea3df8fb5b386722ad2fbd00c0e1c854


In [3]:
!docker context ls

NAME        DESCRIPTION                               DOCKER ENDPOINT               KUBERNETES ENDPOINT   ORCHESTRATOR
default *   Current DOCKER_HOST based configuration   unix:///var/run/docker.sock                         swarm


In [4]:
host_ip = "0.0.0.0"

In [36]:
%%bash -s $host_ip
curl ${1}:8001/list_models

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1156  100  1156    0     0  16542      0 --:--:-- --:--:-- --:--:-- 16753


{"models":[{"model_type":"Claude","model_name":"anthropic.claude-instant-v1","model_family":"bedrock","streaming":true},{"model_type":"Claude","model_name":"anthropic.claude-v1","model_family":"bedrock","streaming":true},{"model_type":"Claude","model_name":"anthropic.claude-v2","model_family":"bedrock","streaming":true},{"model_type":"Titan","model_name":"amazon.titan-tg1-large","model_family":"bedrock","streaming":true},{"model_type":"Jurassic","model_name":"ai21.j2-grande-instruct","model_family":"bedrock","streaming":false},{"model_type":"Jurassic","model_name":"ai21.j2-jumbo-instruct","model_family":"bedrock","streaming":false},{"model_type":"Jurassic","model_name":"ai21.j2-mid","model_family":"bedrock","streaming":false},{"model_type":"Jurassic","model_name":"ai21.j2-mid-v1","model_family":"bedrock","streaming":false},{"model_type":"Jurassic","model_name":"ai21.j2-ultra-v1","model_family":"bedrock","streaming":false},{"model_type":"Cohere","model_name":"cohere.command-text-v14","m

In [37]:
!docker logs test-api-layer

INFO:     Started server process [1]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8001 (Press CTRL+C to quit)
api-layer-ddb
INFO:     172.17.0.1:52678 - "GET /list_models HTTP/1.1" 200 OK


In [33]:
!docker kill test-api-layer && docker rm test-api-layer

test-api-layer
test-api-layer


In [38]:
!aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 705484421587.dkr.ecr.us-west-2.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [39]:
!docker tag api-layer:latest 705484421587.dkr.ecr.us-west-2.amazonaws.com/api-layer:latest

In [40]:
!docker push 705484421587.dkr.ecr.us-west-2.amazonaws.com/api-layer:latest

The push refers to repository [705484421587.dkr.ecr.us-west-2.amazonaws.com/api-layer]

affa9286: Preparing 
8c8dc9cc: Preparing 
57095779: Preparing 
ebb9bed6: Preparing 
039d7570: Preparing 
7097618f: Preparing 
ab009408: Preparing 
ec988e2a: Preparing 
fd493130: Preparing 
96cc1454: Pushed   77.87MB/74.82MBlatest: digest: sha256:7588fb8f22cea879da3f5793ed98d3e4a285edcda8b0cc553d2678090197f7f0 size: 2416
